In [720]:
import json
import numpy as np
import pandas as pd
import pymc3 as pm 
import theano.tensor as tt
import sys
pd.set_option('display.max_columns', None)

In [721]:
import theano
theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"

In [754]:
week_nums

0      40
1      41
2      42
3      43
4      44
       ..
456    27
457    28
458    29
459    30
460    31
Name: week_num, Length: 461, dtype: int64

In [722]:
predictor_df = pd.read_csv('state_stats.csv')
predictor_df.drop(index='FL', inplace=True, errors='ignore')
flu_ground_truth_df = pd.read_csv('flu_ground_truth.csv')
week_nums = flu_ground_truth_df.week_num
flu_ground_truth_df.drop(columns='week_num', inplace=True)

In [723]:
abbrev2state = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state2abbrev = {v: k for k, v in abbrev2state.items()}

In [724]:
flu_ground_truth_df.columns = [state2abbrev[state] for state in flu_ground_truth_df.columns]

In [725]:
flu_ground_truth_df.head()

,TX,LA,WA,NJ,AR,MT,ND,NE,NM,IN,CA,OH,SC,RI,MI,GA,WY,NH,MD,MS,CT,WV,IA,IL,VT,TN,MO,WI,NC,PA,KY,DE,KS,OR,MA,AL,AZ,MN,ME,NV,ID,UT,VA,NY,OK,SD,CO
0,2.06514,2.28108,0.510041,2.31534,0.696056,0.037119,0.671355,2.199780,2.13191,0.518962,1.95412,0.228257,0.446127,0.235036,0.671831,1.60490,0.632911,0.075930,1.31920,2.90980,0.078309,1.59741,0.563063,1.28205,1.47641,0.274655,1.101760,0.465022,0.531430,0.703292,0.142932,0.100125,0.442227,0.551334,0.604621,2.13477,0.674721,0.574800,0.357724,1.66683,0.414938,0.747696,1.14343,0.262990,0.624179,1.102750,0.660684
1,2.09394,2.04776,1.040070,2.47836,0.674157,0.180963,0.721961,2.735320,1.96350,0.653137,2.15266,0.361628,0.765523,0.201545,0.585011,1.71307,0.440621,0.092464,2.09879,2.97075,0.238095,1.58968,0.860956,1.45089,1.35777,0.499492,0.930521,0.581832,0.536530,0.910726,0.223975,0.368550,0.577490,0.624076,0.887866,2.05999,0.749939,0.541293,0.434855,1.63633,1.457980,0.410939,1.23653,0.369729,0.958678,0.793794,0.628621
2,2.10072,2.42974,0.904393,2.24812,0.514217,0.185874,0.567477,3.311030,1.67486,0.584346,2.24173,0.523560,0.389430,0.362845,0.524238,2.01249,0.441798,0.293610,1.45439,2.98141,0.374158,1.52672,0.407000,1.56607,1.48221,0.499550,0.989890,1.188220,0.457107,0.881081,0.156152,0.337025,0.479750,0.484848,0.744806,1.94224,0.953365,0.521162,0.421743,1.50860,1.408140,0.440583,1.22545,0.362121,2.070390,0.791023,0.804020
3,2.20655,1.92854,0.955110,2.79286,0.413650,0.036390,0.339239,2.057610,2.35498,0.581084,1.91748,0.329518,0.418354,0.322675,0.921435,2.38747,0.490305,0.178795,1.26769,3.14804,0.333542,1.81171,0.183225,1.45762,1.44393,0.401638,0.959713,1.010490,0.511163,0.721180,0.150108,0.460494,0.538071,0.751771,0.783721,2.27650,0.888804,0.637716,0.338409,2.14451,1.280070,0.755957,1.26902,0.358981,1.670730,0.801311,0.909658
4,2.36381,2.80934,0.786370,1.82877,1.090280,0.040519,0.713749,0.376712,2.37983,0.842254,2.52326,0.831798,0.445310,0.530256,0.863417,2.65614,0.566636,0.294372,1.45386,3.35570,0.396743,1.83986,0.358780,1.48930,1.25276,0.528096,1.167680,1.096490,0.649925,0.991455,0.289678,0.222332,0.481289,0.764895,0.812421,2.83371,1.185730,0.805802,0.176471,1.76761,1.827240,0.651859,1.26547,0.527357,1.628450,0.779833,0.971705


In [726]:
flu_percent_change_df = pd.DataFrame()

In [727]:
for state in flu_ground_truth_df.columns:
    wili_vals = flu_ground_truth_df[state]
    flu_percent_change_df[state] = [(wili_vals[i+1] - wili_vals[i]) / wili_vals[i] 
                                    for i in range(len(wili_vals) - 1)]

In [728]:
flu_percent_change_df = flu_percent_change_df[predictor_df.index]

In [729]:
flu_percent_change_df.head()

,NY,NJ,PA,IL,MD,MA,VA,CA,RI,MI,TX,MO,MN,CT,GA,OH,CO,DE,IN,UT,KY,NE,TN,OR,LA,NC,OK,WA,KS,WI,NH,AZ,SC,AL,IA,NM,WV,NV,AR,ID,ME,MS,VT,SD,ND,MT,WY
0,0.405867,0.070409,0.294947,0.131695,0.590957,0.468467,0.081422,0.101601,-0.142493,-0.129229,0.013946,-0.155423,-0.058293,2.040475,0.067400,0.584302,-0.048530,2.680899,0.258545,-0.450393,0.567004,0.243452,0.818616,0.131938,-0.102285,0.009597,0.535902,1.039189,0.305868,0.251192,0.217754,0.111480,0.715931,-0.035030,0.529058,-0.078995,-0.004839,-0.018298,-0.031462,2.513730,0.215616,0.020946,-0.080357,-0.280169,0.075379,3.875146,-0.303818
1,-0.020577,-0.092900,-0.032551,0.079386,-0.307034,-0.161128,-0.008961,0.041377,0.800318,-0.103884,0.003238,0.063802,-0.037191,0.571465,0.174786,0.447786,0.279022,-0.085538,-0.105324,0.072137,-0.302815,0.210473,0.000116,-0.223095,0.186536,-0.148031,1.159630,-0.130450,-0.169250,1.042205,2.175391,0.271257,-0.491289,-0.057160,-0.527270,-0.147003,-0.039605,-0.078059,-0.237244,-0.034184,-0.030153,0.003588,0.091650,-0.003491,-0.213978,0.027138,0.002671
2,-0.008671,0.242309,-0.181483,-0.069250,-0.128370,0.052249,0.035554,-0.144643,-0.110708,0.757665,0.050378,-0.030485,0.223643,-0.108553,0.186326,-0.370620,0.131387,0.366350,-0.005582,0.715811,-0.038706,-0.378559,-0.196000,0.550529,-0.206277,0.118257,-0.193036,0.056078,0.121565,-0.149577,-0.391046,-0.067719,0.074273,0.172100,-0.549816,0.406076,0.186668,0.421523,-0.195573,-0.090950,-0.197594,0.055890,-0.025826,0.013006,-0.402198,-0.804222,0.109795
3,0.469039,-0.345198,0.374768,0.021734,0.146858,0.036620,-0.002797,0.315925,0.643313,-0.062965,0.071270,0.216697,0.263575,0.189484,0.112533,1.524287,0.068209,-0.517188,0.449453,-0.137704,0.929797,-0.816918,0.314856,0.017457,0.456719,0.271463,-0.025306,-0.176671,-0.105529,0.085107,0.646422,0.334074,0.064433,0.244766,0.958139,0.010552,0.015538,-0.175751,1.635755,0.427453,-0.478527,0.065965,-0.132396,-0.026804,1.103971,0.113451,0.155681
4,-0.042857,0.204689,0.042680,0.043390,-0.022196,-0.097843,0.115514,0.088124,-0.099674,-0.065760,0.065136,0.048010,0.017769,-0.194537,0.284966,-0.377241,-0.260048,2.574204,-0.107092,0.018472,-0.230197,6.860328,0.124235,0.013610,-0.257559,-0.266281,0.580970,0.357173,0.506357,-0.057146,0.034113,-0.202738,0.147740,0.221805,0.166899,-0.000937,0.124406,-0.544448,0.098213,-0.182187,1.020853,0.171425,-0.100346,-0.046688,0.062152,5.019507,-0.182304


In [515]:
# flu_percent_change_df.to_csv('flu_percent_change.csv', index_label=False)

In [730]:
predictor_df.head()

,density_metric,Latitude,is_coastal,airport_arrivals,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,spring,summer,fall,winter,partisan_score,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,cancer,NY_dest,NJ_dest,PA_dest,IL_dest,MD_dest,MA_dest,VA_dest,CA_dest,RI_dest,MI_dest,TX_dest,MO_dest,MN_dest,CT_dest,GA_dest,OH_dest,CO_dest,DE_dest,FL_dest,IN_dest,UT_dest,KY_dest,NE_dest,TN_dest,OR_dest,LA_dest,NC_dest,OK_dest,WA_dest,KS_dest,WI_dest,NH_dest,AZ_dest,SC_dest,AL_dest,IA_dest,NM_dest,WV_dest,NV_dest,AR_dest,ID_dest,ME_dest,MS_dest,VT_dest,SD_dest,ND_dest,MT_dest,WY_dest,NY_is_neighbor,NJ_is_neighbor,PA_is_neighbor,IL_is_neighbor,MD_is_neighbor,MA_is_neighbor,VA_is_neighbor,CA_is_neighbor,RI_is_neighbor,MI_is_neighbor,TX_is_neighbor,MO_is_neighbor,MN_is_neighbor,CT_is_neighbor,GA_is_neighbor,OH_is_neighbor,CO_is_neighbor,DE_is_neighbor,FL_is_neighbor,IN_is_neighbor,UT_is_neighbor,KY_is_neighbor,NE_is_neighbor,TN_is_neighbor,OR_is_neighbor,LA_is_neighbor,NC_is_neighbor,OK_is_neighbor,WA_is_neighbor,KS_is_neighbor,WI_is_neighbor,NH_is_neighbor,AZ_is_neighbor,SC_is_neighbor,AL_is_neighbor,IA_is_neighbor,NM_is_neighbor,WV_is_neighbor,NV_is_neighbor,AR_is_neighbor,ID_is_neighbor,ME_is_neighbor,MS_is_neighbor,VT_is_neighbor,SD_is_neighbor,ND_is_neighbor,MT_is_neighbor,WY_is_neighbor,overall_vacc_rate,child_vacc_rate
NY,10711.4,40.705626,1,2.625045,0.22,0.09,0.13,0.26,0.14,0.16,43.6,66.5,48.1,23.3,-12,18.737,29.756,51.436,5.347,61.622797,14.719,33.425,24.046,24.675,200.165,0.911538,0.061864,0.004793,0.000216,0.000722,0.001985,0.000495,0.000199,0.001692,0.000196,0.000148,0.000147,0.000207,0.033303,0.000321,0.000261,0.000237,0.002078,0.000570,0.000153,0.000171,0.000122,0.000156,0.000232,0.000098,0.000264,0.000368,0.000089,0.000126,0.000117,0.000155,0.001047,0.000143,0.000251,0.000122,0.000119,0.000088,0.000168,0.000233,0.000099,0.000080,0.000857,0.000131,0.012476,0.000081,0.000000,0.000105,0.000209,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,81.7,69.6
NJ,2789.6,40.143006,1,2.333070,0.23,0.08,0.11,0.27,0.14,0.16,50.6,72.2,54.8,33.0,-7,19.372,27.143,51.199,5.676,72.829584,11.590,36.240,22.909,25.672,200.319,0.028068,0.823014,0.019686,0.000128,0.000793,0.000447,0.000315,0.000072,0.000472,0.000089,0.000099,0.000112,0.000079,0.001903,0.000142,0.000171,0.000154,0.016296,0.000245,0.000081,0.000067,0.000091,0.000065,0.000098,0.000052,0.000080,0.000205,0.000059,0.000067,0.000060,0.000069,0.000368,0.000132,0.000183,0.000051,0.000015,0.000025,0.000137,0.000081,0.000045,0.000041,0.000325,0.000034,0.000566,0.000000,0.000000,0.000114,0.000000,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79.4,72.8
PA,1957.6,40.994593,1,1.621257,0.22,0.08,0.12,0.25,0.14,0.18,47.4,68.6,50.9,28.4,0,6.767,17.371,51.067,5.199,54.960700,11.110,28.104,23.155,29.331,230.835,0.003142,0.028442,0.877457,0.000199,0.012778,0.000454,0.000731,0.000079,0.000352,0.000216,0.000143,0.000188,0.000158,0.000587,0.000222,0.002901,0.000164,0.069599,0.000248,0.000159,0.000125,0.000229,0.000058,0.000202,0.000053,0.000180,0.000314,0.000203,0.000058,0.000106,0.000117,0.000366,0.000156,0.000250,0.000151,0.000091,0.000100,0.011997,0.000114,0.000207,0.000065,0.000299,0.000101,0.000294,0.000080,0.000137,0.000137,0.000232,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,82.5,69.7
IL,1761.9,39.739318,0,4.102417,0.24,0.09,0.12,0.26,0.13,0.15,51.6,73.4,53.8,28.3,-7,16.918,22.729,50.902,5.963,59.266824,12.397,32.078,21.199,27.340,225.471,0.000143,0.000187,0.000201,0.900515,0.000150,0.000173,0.000197,0.000141,0.000231,0.000784,0.000194,0.018036,0.000492,0.000210,0.000285,0.000342,0.000297,0.000204,0.000304,0.015667,0.000116,0.001515,0.000491,0.000354,0.000092,0.000159,0.000204,0.000159,0.000138,0.000288,0.011498,0.000137,0.000223,0.000114,0.000131,0.018256,0.000071,0.000108,0.000336,0.000304,0.000190,

In [731]:
comparison_predictors = ['density_metric', 'Latitude', 'is_coastal', 'airport_arrivals', 'Children 0-18', 
                          'Adults 19-25', 'Adults 26-34', 'Adults 35-54', 'Adults 55-64', '65+', 
                         'partisan_score', 'hispanic', 'minority', 'female', 
                         'income', 'nodegree', 'bachelor', 'inactivity', 'obesity', 'cancer',
                         'overall_vacc_rate', 'child_vacc_rate']
season_predictors = ['spring', 'summer', 'fall', 'winter']
no_comparison_predictors = ['commuters', 'is_neighbor']

comparison_preds_df = predictor_df[comparison_predictors + season_predictors]

In [732]:
def get_season(week):
    if week >= 52 or week < 13:
        return np.array([0, 0, 0, 1])
    if 13 <= week < 26:
        return np.array([1, 0, 0, 0])
    if 26 <= week < 39:
        return np.array([0, 1, 0, 0])
    if 39 <= week < 52:
        return np.array([0, 0, 1, 0])
    raise ValueError

In [733]:
predictor_num = len(comparison_predictors) + len(season_predictors) + len(no_comparison_predictors)
state_num = flu_percent_change_df.shape[1]
comparison_preds_num = len(comparison_predictors)
obs_num = len(flu_percent_change_df)

In [734]:
season_indictor_array = np.zeros((obs_num, state_num - 1, len(season_predictors)))
for i, week_num in enumerate(week_nums[1:]):
    season_indictor_array[i, :, :] = np.repeat(get_season(week_num)[np.newaxis, :], state_num - 1, axis=0)

In [735]:
X = np.zeros((state_num * obs_num, state_num - 1, predictor_num + 1))
Y_target = np.zeros(X.shape[0])
Y_state_idx = np.zeros(X.shape[0], dtype=int)
X_flu = np.zeros((X.shape[0], state_num - 1))
X.shape

(21620, 46, 29)

In [736]:
X_flu.shape

(21620, 46)

In [737]:
for idx, state in enumerate(predictor_df.index):
    
    # response variable
    Y_target[obs_num * idx: obs_num * idx + obs_num] = flu_percent_change_df[state]
    
    # percent change of other states
    X_flu[obs_num * idx: obs_num * idx + obs_num, :] = flu_percent_change_df.drop(columns=state).to_numpy()
    
    Y_state_idx[obs_num * idx: obs_num * idx + obs_num] = [idx] * obs_num
    
    state_comparison_preds = np.array(comparison_preds_df.loc[state])
    
    constant_design_matrix = np.zeros((X.shape[1], X.shape[2]))
    constant_design_matrix[:, 0] = np.ones(state_num - 1)
    
    # two predictors that aren't differences between states: neighboring state and number of commuters
    other_states_preds_df = predictor_df.drop(index=state)
    not_difference_matrix = other_states_preds_df[[state + '_is_neighbor', state + '_dest']].to_numpy()
    constant_design_matrix[:, 1: 1 + len(no_comparison_predictors)] = not_difference_matrix
    
    # the rest of the predictors are differences between two states
    other_states_comparison_preds_array = comparison_preds_df.drop(index=state).to_numpy()
    difference_matrix = abs((other_states_comparison_preds_array - state_comparison_preds) ** 1)
    constant_design_matrix[:, 1 + len(no_comparison_predictors):] = difference_matrix
    
    constant_design_matrix_3D = np.repeat(constant_design_matrix[np.newaxis, :, :], repeats=obs_num, axis=0)
    
    # pick out appropriate season and set the rest of the temperature predictors to zero
    constant_design_matrix_3D[:, :, -len(season_predictors):] *= season_indictor_array 
    
    X[obs_num * idx: obs_num * idx + obs_num, :, :] = constant_design_matrix_3D 

In [738]:
np.random.seed(109)
indices = np.arange(len(Y_target))
np.random.shuffle(indices)
Y_target_random = Y_target[indices]
X_flu_random = X_flu[indices]
X_random = X[indices]
Y_state_idx_random = Y_state_idx[indices]

In [524]:
model = pm.Model()

with model:
    beta = pm.Normal('beta', mu=0, sigma=100, shape=predictor_num + 1)
    sigma_sq = pm.InverseGamma('sigma_sq', alpha=0.1, beta=0.1, shape=state_num)
    nu = pm.Deterministic('nu', pm.math.dot(X_random, beta))
    lambda_ = pm.Deterministic('lambda', pm.math.exp(nu))
    alpha = pm.Exponential('alpha', lam=1/lambda_, shape=(X_random.shape[0], state_num - 1))
    mu = pm.Deterministic('mu', pm.math.sum(alpha * X_flu_random, axis=1) / pm.math.sum(alpha, axis=1))
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=pm.math.sqrt(sigma_sq[Y_state_idx_random]), observed=Y_target_random)

INFO (theano.gof.compilelock): Refreshing lock /Users/wfried/.theano/compiledir_Darwin-18.6.0-x86_64-i386-64bit-i386-3.7.6-64/lock_dir/lock


In [739]:
np.random.seed(109)
betas = np.random.normal(0, 0.5, predictor_num + 1)
true_sigmas = abs(np.random.normal(0, 2, state_num))

In [740]:
normalize = ([1, 0.5, np.mean(list(predictor_df['NY_dest'])[1:])] + 
             [np.mean(predictor_df[pred]) for pred in comparison_predictors + season_predictors])

normalize_array = np.array(normalize)

In [741]:
true_betas = betas / (normalize_array * 10)

In [742]:
np.random.seed(209)
betas_vec = true_betas.reshape(-1, 1)
Y_sim = np.zeros(X_random.shape[0])

for i in range(X_random.shape[0]):
    design_mat = X[i, :, :]
    nu = (design_mat @ betas_vec).flatten()
    lambda_ = np.exp(nu)
    alpha = np.random.exponential(scale=lambda_)
    mu = np.dot(alpha, X_flu[i, :]) / np.sum(alpha)
    Y_sim[i] = np.random.normal(mu, true_sigmas[Y_state_idx_random[i]])

In [743]:
np.mean(Y_sim)

29.474794280063517

In [671]:
np.mean(Y_target)

29.626192401961408

In [686]:
Y_target[3970:4000]

array([-7.30675683e-01, -4.44221162e-02, -9.01047710e-02, -7.53927680e-01,
       -9.98129002e-01,  8.42407000e+02, -9.98814333e-01,  5.57659000e+02,
        3.46830535e-02, -9.98270001e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  3.73672000e+02, -9.97330999e-01,  0.00000000e+00,
        6.63231000e+02,  1.36962743e+00,  7.98709005e+00, -1.25743169e-01,
       -3.23825080e-01, -6.03360153e-01,  2.66323962e-01, -4.51658064e-02,
        7.55387270e-01, -4.19773657e-01,  1.24671331e-01, -1.57970352e-02,
       -2.65716061e-01, -9.87126273e-02])

In [698]:
3970 - 3680

290

In [700]:
flu_ground_truth_df['Rhode Island'][290:310]

290    0.927536
291    0.249808
292    0.238711
293    0.217202
294    0.053447
295    0.000100
296    0.084341
297    0.000100
298    0.055866
299    0.057803
300    0.000100
301    0.000100
302    0.000100
303    0.000100
304    0.037467
305    0.000100
306    0.000100
307    0.066423
308    0.157398
309    1.414550
Name: Rhode Island, dtype: float64

In [753]:
flu_ground_truth_df.apply(lambda x: sum(x == 0.0001))

TX      0
LA      0
WA     12
NJ      0
AR     24
MT    150
ND     79
NE      1
NM      0
IN      0
CA      0
OH      0
SC     10
RI     54
MI      0
GA      0
WY      8
NH     36
MD      1
MS      0
CT     56
WV      0
IA     77
IL      0
VT      2
TN      3
MO     11
WI      1
NC      4
PA      0
KY     11
DE    135
KS      5
OR     23
MA      0
AL      0
AZ      0
MN      0
ME      5
NV      0
ID     61
UT      2
VA      0
NY     14
OK      8
SD      8
CO      2
dtype: int64

In [750]:
# flu_ground_truth_df

for index, row in flu_percent_change_df.iterrows():
    for column in flu_percent_change_df.columns:
        if row[column] > 20:
            print('above', round(flu_ground_truth_df.iloc[index][column], 5))
        if row[column] < -0.95:
            print('below', round(flu_ground_truth_df.iloc[index + 1][column], 5))

below 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001
below 0.0001
below 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
below 0.0001
below 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
above 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
above 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
above 0.0001
above 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
above 0.0001
below 0.0001
below 0.0001
above 0.0001
above 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001
above 0.0001
below 0.0001
below 0.0001

In [707]:
flu_percent_change_df.apply(lambda x: (min(x), max(x)))

NY                (-0.9996943746599919, 1819.39)
NJ      (-0.9357180214313939, 9.997658630072591)
PA      (-0.5520280203246015, 0.713590372696799)
IL    (-0.45923830178759206, 0.6054454895729214)
MD     (-0.9993289986647074, 12.033603707995363)
MA     (-0.4074851494829819, 0.9414801769472425)
VA     (-0.5259302305105041, 0.8121106015477757)
CA      (-0.5905816887547081, 1.452375960613925)
RI     (-0.9995892667178715, 5135.5199999999995)
MI       (-0.7221029491668445, 3.64052818518975)
TX      (-0.4571613068996295, 0.829493580599144)
MO                (-0.9997273332097244, 2339.35)
MN      (-0.9249416089137343, 7.158299385662542)
CT     (-0.9999350834826414, 13087.999999999998)
GA     (-0.7941132584677864, 1.9437740224972726)
OH     (-0.9464746145924475, 12.762178583053055)
CO     (-0.9997978822097943, 7398.2699999999995)
DE      (-0.9998301428498633, 5886.299999999999)
IN     (-0.7803916400508542, 3.8576975307461314)
UT                (-0.9999178469324045, 10067.9)
KY      (-0.99956095